### Student Number : 2
#### Alexandre Lemonnier (alexandre.lemonnier)
#### Victor Simonin (victor.simonin)

First, let's import our useful libraries

In [24]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pandas as pd

In [25]:
spark_application_name = "Projet_LAVS"
spark = (SparkSession.builder.appName(spark_application_name).getOrCreate())

In [26]:
Columns = [StructField("Date",TimestampType()), StructField("High",FloatType()), StructField("Low",FloatType()), StructField("Open",FloatType()), StructField("Close",FloatType()), StructField("Volume",IntegerType()), StructField("Adj Close",FloatType()), StructField("company_name",StringType())]
Schema = StructType(Columns)

def read_data(path):
    return spark.read.schema(Schema).csv(path, header=True, sep=',')

In [29]:
amazon = read_data('data/AMAZON.csv')
apple = read_data('data/APPLE.csv')
facebook = read_data('data/FACEBOOK.csv')
google = read_data('data/GOOGLE.csv')
microsoft = read_data('data/MICROSOFT.csv')
tesla = read_data('data/TESLA.csv')
zoom = read_data('data/ZOOM.csv')

Now let's do the exploration of our datasets :

In [72]:
def exploration(dataset):
    print("Exploration of the dataset", dataset.head().company_name)
    '''
    print("\nThe 40 first elements : ")
    for el in dataset.head(40):
        print(el)
        
    print("\nThe 40 Last elements : ")
    for el in dataset.tail(40):
        print(el)
    ''' 
        
    print("\nThe total number of observations is", dataset.count())
    
    print("\nThe first date is", dataset.head().Date)
    print("The second date is", dataset.head(2)[1].Date)
    print("So we deduce the priod between the data points is a Day")
    
    print("\nSome statistics :")
    print(dataset.summary().show())

In [73]:
exploration(amazon)

Exploration of the dataset AMAZON

The total number of observations is 987

The first date is 2017-01-03 00:00:00
The second date is 2017-01-04 00:00:00
So we deduce the priod between the data points is a Day

Some statistics :
+-------+------------------+------------------+-----------------+------------------+-----------------+------------------+------------+
|summary|              High|               Low|             Open|             Close|           Volume|         Adj Close|company_name|
+-------+------------------+------------------+-----------------+------------------+-----------------+------------------+------------+
|  count|               987|               987|              987|               987|              987|               987|         987|
|   mean|1762.0071216958152|1722.1011452099956|1743.433881363487|1742.9566644206718| 4509728.05775076|1742.9566644206718|        null|
| stddev| 667.2385315752688| 644.7988093382758|657.1153070927137| 655.9576061129322|2179817.62863

In [77]:
from pyspark.sql.functions import isnan, when, count, col

In [78]:
amazon.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in amazon.columns]).show()

AnalysisException: cannot resolve 'isnan(Date)' due to data type mismatch: argument 1 requires (double or float) type, however, 'Date' is of timestamp type.;
'Aggregate [count(CASE WHEN (isnan(Date#136) OR isnull(Date#136)) THEN Date END) AS Date#3456, count(CASE WHEN (isnan(High#137) OR isnull(High#137)) THEN High END) AS High#3458L, count(CASE WHEN (isnan(Low#138) OR isnull(Low#138)) THEN Low END) AS Low#3460L, count(CASE WHEN (isnan(Open#139) OR isnull(Open#139)) THEN Open END) AS Open#3462L, count(CASE WHEN (isnan(Close#140) OR isnull(Close#140)) THEN Close END) AS Close#3464L, count(CASE WHEN (isnan(cast(Volume#141 as double)) OR isnull(Volume#141)) THEN Volume END) AS Volume#3466L, count(CASE WHEN (isnan(Adj Close#142) OR isnull(Adj Close#142)) THEN Adj Close END) AS Adj Close#3468L, count(CASE WHEN (isnan(cast(company_name#143 as double)) OR isnull(company_name#143)) THEN company_name END) AS company_name#3470L]
+- Relation [Date#136,High#137,Low#138,Open#139,Close#140,Volume#14